In [ ]:
using HerbGrammar, HerbData, HerbSearch, HerbInterpret, HerbBenchmarks

import HerbInterpret.interpret

The original benchmark uses a wide set of instructions, which are divided into subgroups. The instructions were originally written in Clojure and can be found [here](https://github.com/thelmuth/Clojush/tree/psb2-v1.0/src/clojush/instructions). 

We translated these to Julia functions using ChatGPT with the prompt: "Can you give me julia functions for the following clojure definitions:". Each of the instruction sets can be found in the `grammars` directory. We create a `cfgrammar` for each set, so they can be imported individually. 

In [ ]:
include("grammars/grammar_char.jl")
include("grammars/grammar_gtm.jl")
include("grammars/grammar_bool.jl")
include("grammars/grammar_numbers.jl")
include("grammars/grammar_string.jl")
include("grammars/grammar_random.jl")

In [ ]:
function make_stacks()
    return Dict(
        :char => [],
        :float => [],
        :integer => [],
        :string => [],
        :boolean => [],
    )
end

g_random = @cfgrammar begin
    State = make_stacks()
    State = boolean_rand(State)
    State = float_rand(State)
    State = integer_rand(State)
    State = string_rand(State)
    State = code_rand(State)
    State = code_rand_atom(State)
    State = char_rand(State)
end

# TODO: Update the rest to match the pattern of g_random

g_char = @cfgrammar begin
    State = make_stacks()
    State = char_allfromstring(State)
    State = char_fromfloat(State)
    State = char_fromfloat(State)
    State = char_isletter(State)
    State = char_isdigit(State)
    State = char_iswhitespace(State)
    State = char_lowercase(State)
    State = char_uppercase(State)
end

g_gtm = @cfgrammar begin
    State = make_stacks()
    State = init_gtm(State)
    State = ensure_instruction_map(State)
    State = load_track(State)
    State = dump_track(State)
    State = trace(State)
    State = gtm_left(State)
    State = gtm_right(State)
    State = gtm_inc_delay(State)
    State = gtm_dec_delay(State)
    State = gtm_dub1(State)
end

g_bool = @cfgrammar begin
    State = make_stacks()
    State = boolean_and(State)
    State = boolean_or(State)
    State = boolean_xor(State)
    State = boolean_not(State)
    State = boolean_invert_first_then_and(State)
    State = boolean_invert_second_then_and(State)
    State = boolean_fromfloat(State)
    State = boolean_fromfloat(State)
end

g_integer = @cfgrammar begin
    State = make_stacks()
    State = integer_add(State)
    State = integer_sub(State)
    State = integer_mult(State)
    State = integer_div(State)
    State = integer_mod(State)
    State = integer_lt(State)
    State = integer_gt(State)
    State = integer_lte(State)
    State = integer_gte(State)
    State = integer_fromboolean(State)
    State = integer_fromfloat(State)
    State = integer_fromstring(State)
    State = integer_fromchar(State)
    State = integer_min(State)
    State = integer_max(State)
    State = integer_inc(State)
    State = integer_dec(State)
    State = integer_abs(State)
    State = integer_negate(State)
    State = integer_pow(State)
end

g_float = @cfgrammar begin
    State = make_stacks()
    State = float_add(State)
    State = float_sub(State)
    State = float_mult(State)
    State = float_div(State)
    State = float_mod(State)
    State = float_lt(State)
    State = float_gt(State)
    State = float_lte(State)
    State = float_gte(State)
    State = float_fromboolean(State)
    State = float_fromfloat(State)
    State = float_fromstring(State)
    State = float_fromchar(State)
    State = float_min(State)
    State = float_max(State)
    State = float_inc(State)
    State = float_dec(State)
    State = float_abs(State)
    State = float_negate(State)
    State = float_pow(State)
    State = float_arctan(State)
    State = float_arccos(State)
    State = float_arcsin(State)
    State = float_floor(State)
    State = float_ceiling(State)
    State = float_log10(State)
    State = float_log2(State)
    State = float_square(State)
    State = float_sqrt(State)
    State = float_tan(State)
    State = float_cos(State)
    State = float_sin(State)
end

g_string = @cfgrammar begin
    Start = State
    State = make_stacks()
    State = string_frominteger(State)
    State = string_fromfloat(State)
    State = string_fromchar(State)
    State = string_fromboolean(State)
    State = string_concat(State)
    State = string_conjchar(State)
    State = string_take(State)
    State = string_substring(State)
    State = string_first(State)
    State = string_last(State)
    State = string_nth(State)
    State = string_rest(State)
    State = string_butlast(State)
    State = string_length(State)
    State = string_reverse(State)
    State = string_parse_to_chars(State)
    State = string_split(State)
    State = string_emptystring(State)
    State = string_containschar(State)
    State = string_indexofchar(State)
    State = string_occurrencesofchar(State)
    State = string_replace(State)
    State = string_replacefirst(State)
    State = string_replacechar(State)
    State = string_replacefirstchar(State)
    State = string_removechar(State)
    State = string_setchar(State)
    State = string_capitalize(State)
    State = string_uppercase(State)
    State = string_lowercase(State)
    State = exec_string_iterate(State)
    State = string_sort(State)
    State = string_includes(State)
    State = string_indexof(State)
end


In [ ]:
ex = :(string_rand(integer_rand(boolean_rand())))

function interpret(tab::SymbolTable, ex::Expr)
    if ex.head == :call
        # println(keys(tab))
        if ex.args[1] in keys(tab)
            if length(ex.args) > 1
                # println("next step")
                return tab[ex.args[1]](interpret(tab, ex.args[2]))
            else
                # println("execute")
                return tab[ex.args[1]](tab)
            end
        else
            throw(ArgumentError("Argument $(ex.args[1]) not present in symbol table."))
        end
    else
        throw(Error("Expression type not supported: $(ex.head)"))
    end
end

interpret(SymbolTable(grammar), ex)

# Small example

This creates a small example of how to use the grammar. We can merge different different grammars together, each should be annotated by its name (e.g., `Dict{Symbol, Any}(:char => SymbolTable(g_char))`).

Then, we create an input dictionary (e.g., `Dict(:input => "a")`), turn it into a state using `init_gtm` and finally merge it with the grammar dictionaries. 

Finally, we create an example of a program and interpret it. 

In [ ]:
# tab_char = Dict{Symbol, Any}(:char => SymbolTable(g_char))
tab_char = SymbolTable(g_char)
tab_gtm = SymbolTable(g_gtm)
tab = merge(tab_char, tab_gtm)
input = Dict(:input => "a")
input = init_gtm(input)
input = merge(tab, input)
stacks = make_stacks()
input = merge(input, stacks)

# println(input[:])
ex = :(char_isletter()) # should be true

interpret(input, ex)

# Test case

We add a test with one of the benchmark problems `coin_sums`. We use the import script to get the data. 

In [ ]:
include("retrieve_all_tasks.jl")
write_psb2_problems_to_file(["coin-sums"], "edge")
include("datasets/coin-sums/coin_sumsdata.jl");

In [ ]:
tab = merge(SymbolTable(g_gtm), merge(SymbolTable(g_integer), merge(SymbolTable(g_string), SymbolTable(g_char))))
stacks = make_stacks()
tab = merge(tab, stacks)
tab
ex = problem_coin_sums.examples[1]

# res = interpret(tab, ex)
# println("Res: ", res)
